In [1]:
using DynamicPolynomials,LinearAlgebra
using JuMP
import HiGHS

# Step 1 Match Polynomials
$b*monomials=b_1*f_1 + b_2*f_2...$

In [9]:
function getLP(poly,arrayOfPolynomials,var)
    d = maxdegree(poly)
    base=monomials(var,d)
    b=coefficients(poly,base)
    c=[]
    nVars=[]
    for f in arrayOfPolynomials
        deg=d - maxdegree(f)
        push!(nVars,size(monomials(var,deg))[1])
        l = []
        for g in monomials(var,deg)
            basepoly=g*f
            push!(l,coefficients(basepoly,monomials(var,d)))
        end
        l=transpose(reduce(vcat,transpose.(l)))
        push!(c,l)
    end
    c=transpose(reduce(vcat,transpose.(c)))
    
    #Create Linear Program
    m = size(b,1) #Number of Monomials in Polynom to Match
    LP = Model(HiGHS.Optimizer) # Initialize Model
    set_optimizer_attribute(LP, "log_to_console", "false") # disable debug info
    n=size(c,2) # Number of Variables
    @variable(LP, u[1:n]>=0)
    for i in 1:m
        @constraint(LP, sum(dot(u,c[i,:])) == b[i])
    end
    # print(LP)
    return (LP,u,b,c,nVars)
end

getLP (generic function with 1 method)

In [3]:
function getSolution(poly,arrayOfPolynomials,var)
    LP,u,b,c,n = getLP(poly,arrayOfPolynomials,var)
    print(LP)
    JuMP.optimize!(LP)
    if JuMP.has_values(LP)
        sol=JuMP.value.(u)
        walk=1
        c=[]
        solPoly=[]
        for (j,i) in enumerate(n)
            coeff = view(sol,walk:walk+i-1)
            base = monomials(x,maxdegree(e)-maxdegree(F[j]))
            push!(solPoly,polynomial(coeff,base))
            push!(c,coeff)
            walk+=i
        end
        return solPoly
    else
        return false
    end
end

getSolution (generic function with 1 method)

## Example Functions

In [4]:
@polyvar x[1:3]
h = 3x[1] - 2x[2] - 2x[3]
f1 = 1 + 0*x[1]
f2 = x[1]-x[2]
f3 = x[1]-x[3]
f4 = x[1]^2 - 4x[2]x[3]
F=[f1,f2,f3,f4]
e=polynomial(monomials(x,1))*h

3x₁² + x₁x₂ + x₁x₃ - 2x₂² - 4x₂x₃ - 2x₃²

In [5]:
polynomial(monomials(x,1))^4*h

3x₁⁵ + 10x₁⁴x₂ + 10x₁⁴x₃ + 10x₁³x₂² + 20x₁³x₂x₃ + 10x₁³x₃² - 5x₁x₂⁴ - 20x₁x₂³x₃ - 30x₁x₂²x₃² - 20x₁x₂x₃³ - 5x₁x₃⁴ - 2x₂⁵ - 10x₂⁴x₃ - 20x₂³x₃² - 20x₂²x₃³ - 10x₂x₃⁴ - 2x₃⁵

In [6]:
function Run(poly,arrayOfPolynomials)
    var = variables([arrayOfPolynomials...,poly])
    for i in 1:5
        e=polynomial(monomials(x,1))^i*h
        Sol = getSolution(e,arrayOfPolynomials,var)
        if Sol != false
            return Sol
        end
    end    
    return false
end

Run (generic function with 1 method)

In [7]:
Run(h,F)

4-element Vector{Any}:
 0.0
 x₁ + 2.0x₂
 x₁ + 2.0x₃
 1.0

In [8]:
F

4-element Vector{Polynomial{true, Int64}}:
 1
 x₁ - x₂
 x₁ - x₃
 x₁² - 4x₂x₃